# 0 app store crawling

In [1]:
import os
os.getcwd()

'C:\\Users\\LG\\zepeto'

In [2]:
# 핑크색 warning 안내문 방지 
import warnings
warnings.filterwarnings('ignore')

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# 1 크롤링에 필요한 라이브러리를 불러옵니다

In [4]:
import chromedriver_autoinstaller
import time
from selenium import webdriver
from bs4 import BeautifulSoup as bs
import csv

# 2 국내 유저 리뷰 크롤링 

####  apple appstore는 크롤링을 할 수 있는 api를 제공하고 있다 
https://www.apptweak.io/documentation/ios/application_reviews_filtered

#### 제페토 apple appstore id
https://apps.apple.com/app/id1350301428
#### 제페토 googelp play id
https://play.google.com/store/apps/details?id=me.zepeto.main

#### API KEY 
gMrJyjWiD2lc9wLCJTZxs9xUAl0

In [ ]:
curl -X GET -H "X-Apptweak-Key: gMrJyjWiD2lc9wLCJTZxs9xUAl0" https://api.apptweak.com/ios/applications/1350301428/reviews/filter/list.json?country=kr

## 2.1 RSS를 이용하여 앱스토어 리뷰 크롤링 (XML 형식)

In [12]:
# 모듈 로딩

import pandas as pd #크롤링한 데이터를 데이터프레임으로 담아줄 것

import xmltodict #xml 파싱을 위한 모듈
from urllib.request import urlopen # url을 이용하여 웹상의 데이터를 가져올 irllib.request
import json #xmltodict의 결과물을 dict로 바꾸기 위함 

### 2.1.1 Get Data from URL (page 1 파싱)

In [15]:
# 하단 url로 접속해서 xml데이터를 파싱할 것임 
#reopen 함수를 사용하여 해당urk이 가지는 값을 가져온다
url = 'https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml'
response = urlopen(url).read()
response

b'<?xml version="1.0" encoding="utf-8"?>\n\n\t<feed xmlns:im="http://itunes.apple.com/rss" xmlns="http://www.w3.org/2005/Atom" xml:lang="ko">\n\t\t<id>https://mzstoreservices-int-st.itunes.apple.com/kr/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml</id><title>iTunes Store:\xec\x82\xac\xec\x9a\xa9\xec\x9e\x90 \xed\x9b\x84\xea\xb8\xb0</title><updated>2022-07-24T01:28:46-07:00</updated><link rel="alternate" type="text/html" href="https://apps.apple.com/WebObjects/MZStore.woa/wa/viewGrouping?cc=kr&amp;id=29141"/><link rel="self" href="https://mzstoreservices-int-st.itunes.apple.com/kr/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml"/><link rel="first" href="https://itunes.apple.com/kr/rss/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/page=1/id=1350301428/sortby=mostrecent/xml"/><link rel="last" href="h

### 2.1.2 Parsing XML

In [16]:
xml = xmltodict.parse(response)
XmlToJson = json.loads(json.dumps(xml))

### 2.1.3 파싱한 xml 데이터를 내가 원하는대로 바꾸기

In [31]:
res_dict = []

for i in range(len(XmlToJson['feed']['entry'])):
    res_dict.append({
        'DATE' : XmlToJson['feed']['entry'][i]['updated'],
        'STAR' : XmlToJson['feed']['entry'][i]['im:rating'],
        'LIKE' : XmlToJson['feed']['entry'][i]['im:voteSum'],
        'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
        'TITLE' : XmlToJson['feed']['entry'][i]['title'],
        'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
    })

In [32]:
# 잘 변환 되었는지 확인
res_dict

[{'DATE': '2022-07-22T16:51:14-07:00',
  'STAR': '5',
  'LIKE': '0',
  'DISLIKE': 0,
  'TITLE': '너무 고마워요…💗',
  'REVIEW': '으아아아아아ㅏㅏㅏㅏ\n드디어 크리템이!!!!!!!!!!\n아아아아 고마워요오오오오❤️'},
 {'DATE': '2022-07-22T11:25:05-07:00',
  'STAR': '1',
  'LIKE': '0',
  'DISLIKE': 0,
  'TITLE': '후..',
  'REVIEW': '아니 31일째인데 선물이 안보내져요. 제가 왠만하면 말 안하는데 저도 사정이 있고, 이건 아닌것 같습니다. 계속 회원가입 후 30일 어쩌고 진짜 어이없네요'},
 {'DATE': '2022-07-22T07:54:59-07:00',
  'STAR': '5',
  'LIKE': '0',
  'DISLIKE': 0,
  'TITLE': '허위신고, 오류, 불찰도 검토 해주세요😥',
  'REVIEW': '저는 제페토 초창기 때부터 열심히 꾸준히 캐릭터를 키워 온 유저입니다. 저는 신고 먹을 일도 딱히 만든 적이 없고, 조용조용 평범하게 게임을 즐겼는데요. 오늘도 어김없이 월드에서 놀다 갑자기 로그아웃 되더니 서비스 이용 정지 안내라며 며칠 경고도 없이 그냥 바로 영구정지가 되버렸더라고요… 너무너무 놀랐고 상당히 당황스러웠습니다. 제가 잘못하거나 찔리는 게 있다면 그냥 넘어갔겠지만 정말 왜 정지가 됐는지, 혹시 오류나 단순한 불찰인건지 싶어 리뷰와 문의를 드립니다. 저는 이 게임에 제 재산을 탈탈 쏟아부어 약 20만원이 넘게 현질을 한 사람으로써 정말 속상하고 손발이 달달 떨리네요… 제페토 아이디는 churr._. 이고, 하루빨리 발 빠른 조치 취해주셨으면 하고 부탁드립니다. 제페토 언제나 재밌게 운영해주셔서 감사드려요🙇🏻\u200d♀️'},
 {'DATE': '2022-07-22T07:13:59-07:00',
  'STAR': '1',
  'LIKE': '0

In [37]:
#ㅔpage 1 파싱 내용을 데이터프레임으로 담아줍니다 
res_df = pd.DataFrame(res_dict)
res_df

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-07-22T16:51:14-07:00,5,0,0,너무 고마워요…💗,으아아아아아ㅏㅏㅏㅏ\n드디어 크리템이!!!!!!!!!!\n아아아아 고마워요오오오오❤️
1,2022-07-22T11:25:05-07:00,1,0,0,후..,"아니 31일째인데 선물이 안보내져요. 제가 왠만하면 말 안하는데 저도 사정이 있고,..."
2,2022-07-22T07:54:59-07:00,5,0,0,"허위신고, 오류, 불찰도 검토 해주세요😥",저는 제페토 초창기 때부터 열심히 꾸준히 캐릭터를 키워 온 유저입니다. 저는 신고 ...
3,2022-07-22T07:13:59-07:00,1,0,0,ㅋ,ㅈㄴ 잼민이들만 하는 게임임 \n이거 왜함 장애새끼들아 ㅋㅋㅋㅋ\n잼민이들아 걍 쳐...
4,2022-07-22T02:14:56-07:00,5,0,0,좋음,갑자기 제페토 메세지가 다 지워질수있나요?\n지운건 아닌데 다 날아가는거라고 해야하...
5,2022-07-21T23:46:51-07:00,3,0,0,개선할 점,제페토를 잘 사용하고있던 사람입니다 일단 제페토의 좋은점은 날로 사람들이 불편하다 ...
6,2022-07-21T20:35:00-07:00,5,0,0,제페토,고급지고 계산하는 능력이 발달되요
7,2022-07-21T10:28:44-07:00,3,0,0,다크모드 지원 요청,제페토 오랜 유저 입니당 거의 초창기때 부터 재밌게 잘 즐기고 있구 오래전부터 꼭 ...
8,2022-07-21T05:57:50-07:00,4,0,0,젬이 너무 많이 들어요 ㅜ,안녕하세요👋🏻 제페토 유저입니다! 제페토를 하면서 정말 재미있고 완벽한 게임 같아요...
9,2022-07-21T05:25:32-07:00,2,0,0,비번 까먹은 후,안녕하세요 제가 전에 잠깐 지워서 다시 깔았다가 하려구 이메일 클릭했는데 비번을 까...


In [40]:
#시계열 컬럼을 예쁘게 수정해주기
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
res_df.head()

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-07-22 16:51:14,5,0,0,너무 고마워요…💗,으아아아아아ㅏㅏㅏㅏ\n드디어 크리템이!!!!!!!!!!\n아아아아 고마워요오오오오❤️
1,2022-07-22 11:25:05,1,0,0,후..,"아니 31일째인데 선물이 안보내져요. 제가 왠만하면 말 안하는데 저도 사정이 있고,..."
2,2022-07-22 07:54:59,5,0,0,"허위신고, 오류, 불찰도 검토 해주세요😥",저는 제페토 초창기 때부터 열심히 꾸준히 캐릭터를 키워 온 유저입니다. 저는 신고 ...
3,2022-07-22 07:13:59,1,0,0,ㅋ,ㅈㄴ 잼민이들만 하는 게임임 \n이거 왜함 장애새끼들아 ㅋㅋㅋㅋ\n잼민이들아 걍 쳐...
4,2022-07-22 02:14:56,5,0,0,좋음,갑자기 제페토 메세지가 다 지워질수있나요?\n지운건 아닌데 다 날아가는거라고 해야하...


### 2.1.4 반복문으로 page 1~ 10 까지 한번에 긁어오기

In [55]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/kr/rss/customerreviews/page=%i/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/zepeto_kr_review.csv', index = False, encoding = 'utf-8')

In [57]:
#잘 저장 되었는지 확인하기
df = pd.read_csv('data/zepeto_kr_review.csv', encoding = 'utf-8')
df.head()

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-07-22 16:51:14,5,0,0,너무 고마워요…💗,으아아아아아ㅏㅏㅏㅏ\n드디어 크리템이!!!!!!!!!!\n아아아아 고마워요오오오오❤️
1,2022-07-22 11:25:05,1,0,0,후..,"아니 31일째인데 선물이 안보내져요. 제가 왠만하면 말 안하는데 저도 사정이 있고,..."
2,2022-07-22 07:54:59,5,0,0,"허위신고, 오류, 불찰도 검토 해주세요😥",저는 제페토 초창기 때부터 열심히 꾸준히 캐릭터를 키워 온 유저입니다. 저는 신고 ...
3,2022-07-22 07:13:59,1,0,0,ㅋ,ㅈㄴ 잼민이들만 하는 게임임 \n이거 왜함 장애새끼들아 ㅋㅋㅋㅋ\n잼민이들아 걍 쳐...
4,2022-07-22 02:14:56,5,0,0,좋음,갑자기 제페토 메세지가 다 지워질수있나요?\n지운건 아닌데 다 날아가는거라고 해야하...


In [53]:
df['DATE'].sort_values(ascending = True)

499    2022-04-07 04:57:30
498    2022-04-07 04:58:35
497    2022-04-07 04:58:56
496    2022-04-07 08:11:49
495    2022-04-07 10:25:11
              ...         
4      2022-07-22 02:14:56
3      2022-07-22 07:13:59
2      2022-07-22 07:54:59
1      2022-07-22 11:25:05
0      2022-07-22 16:51:14
Name: DATE, Length: 500, dtype: object

# 3 해외 유저 리뷰 크롤링

## 3.1 미국(US) 서버 크롤링

In [58]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/us/rss/customerreviews/page=%i/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/zepeto_us_review.csv', index = False, encoding = 'utf-8')

In [59]:
#잘 저장 되었는지 확인하기
df = pd.read_csv('data/zepeto_us_review.csv', encoding = 'utf-8')
df.head()

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-07-22 17:24:05,5,0,0,Can you?,So the game is good but when I go to the photo...
1,2022-07-22 15:14:36,5,0,0,ZEPETO,Omg this game is so so so sooo amazing. I love...
2,2022-07-22 13:20:43,1,0,0,Do not play this at 3 AM,OK it’s the worst game so I was getting a snac...
3,2022-07-22 11:51:14,3,0,0,I like,But I don’t put in my email in stuff so a 2
4,2022-07-22 10:04:01,5,0,0,About Zepeto,This game is fun


## 3.2 일본(JP) 서버 크롤링

In [61]:
res_dict = []
for j in range(1,11):
    url = 'https://itunes.apple.com/jp/rss/customerreviews/page=%i/id=1350301428/sortby=mostrecent/xml?urlDesc=/customerreviews/id=1350301428/sortBy=mostRecent/xml' % j
    response = urlopen(url).read()

    xml = xmltodict.parse(response)
    XmlToJson = json.loads(json.dumps(xml))

    for i in range(len(XmlToJson['feed']['entry'])):
        res_dict.append({
            'DATE' : XmlToJson['feed']['entry'][i]['updated'],
            'STAR' : int(XmlToJson['feed']['entry'][i]['im:rating']),
            'LIKE' : int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'DISLIKE' : int(XmlToJson['feed']['entry'][i]['im:voteCount']) - int(XmlToJson['feed']['entry'][i]['im:voteSum']),
            'TITLE' : XmlToJson['feed']['entry'][i]['title'],
            'REVIEW' : XmlToJson['feed']['entry'][i]['content'][0]['#text']
        })

res_df = pd.DataFrame(res_dict)
res_df['DATE'] = pd.to_datetime(res_df['DATE'], format="%Y-%m-%dT%H:%M:%S-07:00")
#csv로 저장
res_df.to_csv('data/zepeto_jp_review.csv', index = False, encoding = 'utf-8')

In [62]:
#잘 저장 되었는지 확인하기
df = pd.read_csv('data/zepeto_jp_review.csv', encoding = 'utf-8')
df.head()

,DATE,STAR,LIKE,DISLIKE,TITLE,REVIEW
0,2022-07-22 15:38:29,4,0,0,謎です……,運営様方素敵なアップデートなどをいつもありがとうございます。 クリエイターが送れるようにな...
1,2022-07-22 08:16:58,4,0,0,こんな機能があったら…！,運営さん いつもお疲れ様です( ˆ꒳ˆ )\n今日は、ZEPETOをやっていて\n『こんな機...
2,2022-07-22 07:37:54,4,0,0,お願い、,マッチングの復活求みます！ マッチングで友達を作ってたのでとても悲しいです😢 いろんな方に僕...
3,2022-07-22 06:39:01,1,0,0,全然楽しくない,まず初めに一番嫌なことを紹介します\n1・エロい話が多すぎる\nこれはどう見てもやばいですよ...
4,2022-07-22 05:49:53,1,0,0,^ ^,昔のゼペットに戻して欲しい。\nワールドが多すぎて公式のワールドが昔は満遍なくいたのに今は誰...
